# This is tutorial using TabNetClassifier 
    https://github.com/dreamquark-ai/tabnet (orginal code information)
    This includes Cross Validation test and Hyper parameter tuning
    this code is only for classifcation at the moment (soon I will make tutorial for regressor)


## import libraries

    set target feature and unused function.
    All features need to be float. 

In [66]:
from torch import nn
import torch
from torch import tensor 
from torch.autograd import Variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import torch
import itertools
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm

train_data= pd.read_csv('./data/1.SuicidalideationPassive_rsfmri_train.csv')
test_data= pd.read_csv('./data/1.SuicidalideationPassive_rsfmri_test.csv')
target ='Suicidalideation.x'
unused_feat = ['Set', 'ideation_attempt.x', 'subjectkey', 'abcd_site', 'Unnamed: 0', 'kflod',
               'src_subject_id','interview_date','gender','eventname'
]
Num_FOLDS  = 3
# the number of feature that you want to show 
Num_feat = 5

In [67]:
train_data_processed = train_data.fillna(0).reset_index(drop=True)

features = [ col for col in train_data_processed.columns if col not in unused_feat + [target]] 


In [68]:
train_data_processed[target].dtypes

dtype('int64')

In [69]:
j=0
for i in test_data[features].dtypes:
    #print(j)
    if i == 'object':
        print(test_data[features].columns[j])
    j+=1

## Define preproecessing: fillna as 0, set kflod

In [70]:
def preproecessing (train_data, test_data, NUM_FOLDS):
    test_data_processed= test_data.fillna(0).reset_index(drop=True)
    train_data_processed = train_data.fillna(0).reset_index(drop=True)
    
    test_data_processed["kfold"] = -1

    train_data_processed["kfold"] = -1

    train_data_processed = train_data_processed.sample(frac=1,random_state=2020).reset_index(drop=True)

    kf = KFold(n_splits=NUM_FOLDS)

    for fold, (trn_, val_) in enumerate(kf.split(X=train_data_processed, y=train_data_processed)):
        train_data_processed.loc[val_, 'kfold'] = fold
    print("done preprocessing")
    return train_data_processed, test_data_processed

## Define function of finding best hyperparameters

In [71]:
# Augmented
import torch
import itertools
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm


def find_bestpar(X_train, Y_train, X_valid, Y_valid, X_test, Y_test):
    # Store maximum auc
    max_auc= 0
    # Store maximum hypterparameter set
    max_hy = []
    # define hyperparameter space
  
    n_ = [4,8,16]
    lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4]
    w_ = [0.01, 0.001, 0.0001]
    g_ = [0.95, 0.99, 0.9]
    ss_ = [10, 20, 30]
    # Orginal hyperparameter space 
    """
    # define hyperparameter space (quick version)
    n_ = [4]
    lr_ = [2e-2]
    w_ = [0.01]
    g_ = [0.95]
    ss_ = [10, 20]
    """
    all_ = [n_, lr_, w_, g_, ss_]
    h_space = [s for s in itertools.product(*all_)]
    
    for hy in tqdm(h_space):
        clf = TabNetClassifier(n_a = hy[0],
                               n_d = hy[0],
                               optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                               scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                               scheduler_fn=torch.optim.lr_scheduler.StepLR,
                               verbose=0
                               )
        clf.fit(X_train, Y_train, X_valid, Y_valid, patience=10, batch_size=256, max_epochs=200)
        #clf.fit(X_train, Y_train, X_valid, Y_valid, patience=50, batch_size=256, max_epochs=200) (Recommended)
        preds_acc = clf.predict(X_test)
        preds_prob = clf.predict_proba(X_test)
        test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=Y_test)
        test_acc = accuracy_score(preds_acc, Y_test)
        print("--------Test auc: {} Test acc:{}-------------".format(test_auc, test_acc) )
        if test_auc>max_auc:
            max_hy = hy
            max_auc = test_auc
    return max_hy

## Do it with best hyperparameter 

In [72]:
def bestpar_tuning(X_train, Y_train, X_valid, Y_valid, X_test, Y_test, max_hy):
 
    hy = max_hy
    clf = TabNetClassifier(n_a = hy[0],
                           n_d = hy[0],
                           optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                           scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                           scheduler_fn=torch.optim.lr_scheduler.StepLR,
                           verbose=0
                               )
    clf.fit(X_train, Y_train, X_valid, Y_valid, patience=50, batch_size=256, max_epochs=200)
    preds_acc = clf.predict(X_test)
    preds_prob = clf.predict_proba(X_test)
    test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=Y_test)
    test_acc = accuracy_score(preds_acc, Y_test)
    print("Max hy:" ,hy)
    print("FINAL TEST SCORE(max) test auc: {} test acc: {}".format(test_auc, test_acc))
    return test_auc, clf
    
          

## Find importance feature 

In [73]:
def feature(Num_feat, clf, test_data_processed, features):
    importance =clf.feature_importances_
    plt.plot(importance)
    plt.show()
    labels_importance=importance.argsort()[::-1]

    importance_sort = np.sort(importance)[::-1]

    feat_name_sort=test_data_processed[features].columns[labels_importance]
    important_features = pd.DataFrame() 


    for i in range (Num_feat):
        feature = pd.DataFrame([[feat_name_sort[i],importance_sort[i]]], columns = ['feature name', 'ratio'])
        important_features=pd.concat([important_features,feature])

    print(important_features.reset_index(drop=True))

## Define run(find best par, learn with best par, show feature) function 
    set X_test, Y_test, X_train, Y_train, X_valid, Y_valid

In [74]:

def run(train_data_processed, test_data_processed, fold, Num_feat, features):
    X_test = test_data_processed[features].values
    Y_test = test_data_processed[target].values

    df_train = train_data_processed[train_data_processed.kfold != fold]
    df_valid = train_data_processed[train_data_processed.kfold == fold]
    
    X_train = df_train[features].values
    Y_train = df_train[target].values
    
    X_valid = df_valid[features].values
    Y_valid = df_valid[target].values
    
    y_oof = np.zeros((df_valid.shape[0],len(target)))   # Out of folds validation
    
    print("--------Training Begining for fold {}-------------".format(fold+1))
    n_ = [4,8,16]
    lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4]
    w_ = [0.01, 0.001, 0.0001]
    g_ = [0.95, 0.99, 0.9]
    ss_ = [10, 20, 30]
    all_ = [n_, lr_, w_, g_, ss_]
    h_space = [s for s in itertools.product(*all_)]
    # Start training
    max_hy = find_bestpar(X_train, Y_train, X_valid, Y_valid, X_test, Y_test)
    print("Found maximum hyperparmeter, now work with that")
    test_auc, clf = bestpar_tuning(X_train, Y_train, X_valid, Y_valid, X_test, Y_test, max_hy)
    
    feature(Num_feat, clf, test_data_processed, features)
    
    return test_auc
    
    # VISUALIZTION
    #plt.figure(figsize=(12,6))
    #plt.plot(model.history['train']['loss'])
    #plt.plot(model.history['valid']['loss'])
    
    #Plotting Metric
    #plt.plot([-x for x in model.history['train']['metric']])
    #plt.plot([-x for x in model.history['valid']['metric']])

## Definde function for Cross Validation test 

In [77]:
def hyperparametertuning_CV (train_data, test_data, target, unused_feat, Num_FOLDS, Num_feat):
    CV_auc_all = []
    train_data_processed, test_data_processed = preproecessing (train_data, test_data, Num_FOLDS)
    
    features = [ col for col in train_data_processed.columns if col not in unused_feat + [target]] 
    
    for i in range(Num_FOLDS):
        test_auc = run(train_data_processed, test_data_processed, i, Num_feat, features)
        print ("--------Test auc: {} for fold {}-------------".format(test_auc, i+1))
        print ("                                             ")
        print ("---------------------------------------------")
        CV_auc_all.append(test_auc)
    print(CV_auc_all)
    print("CV mean Test AUC = ", np.mean(CV_auc_all))


## Finally! you can do hyperparametertuning with Cross validation


In [ ]:
"""
train_data= pd.read_csv('./data/train.ct.csv')
test_data= pd.read_csv('./data/test.ct.csv')
target ='sex'
unused_feat = ['Set', 'subjectkey', 'race.ethnicity', 'abcd_site', 'Unnamed: 0', 'fsqc_qc.y', 'kflod']
Num_FOLDS  = 5
# the number of feature that you want to show 
Num_feat = 5

"""    

hyperparametertuning_CV (train_data, test_data, target, unused_feat, Num_FOLDS, Num_feat)


done preprocessing
--------Training Begining for fold 1-------------


Device used : cuda
--------Test auc: 0.7889153719397363 Test acc:0.4796747967479675-------------
Device used : cuda
--------Test auc: 0.7810145951035782 Test acc:0.4796747967479675-------------
Device used : cuda
--------Test auc: 0.7810145951035782 Test acc:0.4796747967479675-------------
Device used : cuda
--------Test auc: 0.7693032015065913 Test acc:0.4796747967479675-------------
Device used : cuda
--------Test auc: 0.7733050847457628 Test acc:0.4959349593495935-------------
Device used : cuda
--------Test auc: 0.7810145951035782 Test acc:0.4796747967479675-------------
Device used : cuda
--------Test auc: 0.7815736817325799 Test acc:0.4796747967479675-------------
Device used : cuda
--------Test auc: 0.7810145951035782 Test acc:0.4796747967479675-------------
Device used : cuda
--------Test auc: 0.7810145951035782 Test acc:0.4796747967479675-------------
Device used : cuda
--------Test auc: 0.7718926553672316 Test acc:0.4796747967479675-------------
Device used : cuda
--------Tes